## Pose Graph Optimization with GTSAM
This notebook demonstrates a simple pose graph optimization (PGO) using the [GTSAM](https://gtsam.org) library.

In [ ]:
import gtsam
from gtsam import Pose2, PriorFactorPose2, BetweenFactorPose2, NonlinearFactorGraph, Values, noiseModel
from gtsam.symbol_shorthand import X
import numpy as np

# Build factor graph
graph = NonlinearFactorGraph()

# Prior on the first pose
prior_model = noiseModel.Diagonal.Sigmas(np.array([0.01, 0.01, 0.01]))
graph.add(PriorFactorPose2(X(0), Pose2(0, 0, 0), prior_model))

# Odometry measurements
odom_model = noiseModel.Diagonal.Sigmas(np.array([0.1, 0.1, 0.1]))
graph.add(BetweenFactorPose2(X(0), X(1), Pose2(1, 0, 0), odom_model))
graph.add(BetweenFactorPose2(X(1), X(2), Pose2(1, 0, 0), odom_model))

# Loop closure measurement
loop_model = noiseModel.Diagonal.Sigmas(np.array([0.1, 0.1, 0.1]))
graph.add(BetweenFactorPose2(X(2), X(0), Pose2(-2, 0, 0), loop_model))

# Initial estimate intentionally off
initial = Values()
initial.insert(X(0), Pose2(0, 0, 0))
initial.insert(X(1), Pose2(2, 0, 0))
initial.insert(X(2), Pose2(4, 0, 0))

print('Initial error:', graph.error(initial))
optimizer = gtsam.GaussNewtonOptimizer(graph, initial)
result = optimizer.optimize()
for i in range(3):
    pose = result.atPose2(X(i))
    print(f'Pose {i}:', pose)
print('Final error:', graph.error(result))

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

G = nx.Graph()
poses = ['X0', 'X1', 'X2']
G.add_nodes_from(poses)
G.add_edges_from([('X0', 'X1'), ('X1', 'X2'), ('X2', 'X0')])

pos = {'X0': (0, 0), 'X1': (1, 0), 'X2': (2, 0)}
edge_labels = {('X0', 'X1'): 'odom', ('X1', 'X2'): 'odom', ('X2', 'X0'): 'loop'}

nx.draw(G, pos, with_labels=True, node_color='lightblue')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
plt.axis('equal')
plt.show()
